In [29]:
!pip install selenium yfinance

# PARTE 1
## Scrape Top 50 Gainers

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [3]:
# Configura el driver de Chrome
driver = webdriver.Chrome()

url = "https://finance.yahoo.com/markets/stocks/gainers"
driver.get(url)
driver.maximize_window()

# Esperamos que cargue la página
wait = WebDriverWait(driver, 240)

# Selector del botón de "Rows per page"
rows_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button.tertiary-btn.menuBtn")))
rows_button.click()

# Esperar a que aparezca la opción de 50 filas y hacer clic
option_50 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,"div[role='option'][data-value='50']")))
option_50.click()

# Esperar que la tabla se actualice con exactamente 50 filas
wait.until(lambda driver: len(driver.find_elements( By.CSS_SELECTOR, "tr[data-testid='data-table-v2-row']")) >= 50)

# Encontrar todas las filas de la tabla
rows = driver.find_elements(By.CSS_SELECTOR, 'tr[data-testid="data-table-v2-row"]')

data = []

# Recorremos las filas y extraemos el ticker y el companyshortname
for row in rows:
    try:
        ticker = row.find_element(By.CSS_SELECTOR, 'td[data-testid-cell="ticker"]').text.strip()
    except:
        ticker = None
    try:
        company = row.find_element(By.CSS_SELECTOR, 'td[data-testid-cell="companyshortname.raw"]').text.strip()
    except:
        company = None

    data.append({"Ticker": ticker, "Company": company})

# Convertimos a DataFrame
df = pd.DataFrame(data)
df.head(50)

,Ticker,Company
0,AMBA,"Ambarella, Inc."
1,IREN,IREN Limited
2,BABA,Alibaba Group Holding Limited
3,AFRM,"Affirm Holdings, Inc."
4,DOOO,BRP Inc.
5,ADSK,"Autodesk, Inc."
6,CIFR,Cipher Mining Inc.
7,SATS,EchoStar Corporation
8,S,"SentinelOne, Inc."
9,EQX,Equinox Gold Corp.


In [4]:
df.to_csv(r"D:\Tarea\datos_stocks.csv", index=False)

# PARTE 2
## Historical Data Retrieval

In [5]:
import yfinance as yf

tickers = df["Ticker"].tolist()

adj_close_df = yf.download(
    tickers=tickers,
    period="1y",
    interval="1mo",
    group_by='ticker',
    auto_adjust=False,
    threads=True
)

# Si hay un MultiIndex, extraemos solo 'Adj Close'
if isinstance(adj_close_df.columns, pd.MultiIndex):
    adj_close_df = adj_close_df.xs('Adj Close', axis=1, level=1)

adj_close_df=adj_close_df.iloc[:-1,:]
adj_close_df.head(20)



[*********************100%***********************]  50 of 50 completed


Ticker,OS,COO,CELH,ADSK,KGC,HMY,GH,SJM,DOOO,EQX,...,SBSW,UPWK,NG,FSM,WPM,EMN,BF-B,BABA,HP,BIDU
Date,,,,,,,,,,,,,,,,,,,,,
2024-09-01,33.900002,110.339996,31.360001,275.480011,9.273230,10.042464,22.940001,116.389412,58.766903,6.09,...,4.11,10.450000,4.10,4.63,60.682819,107.977699,48.019775,104.356186,29.118042,105.290001
2024-10-01,29.520000,104.680000,30.080000,283.799988,9.986554,10.704062,21.879999,109.094666,48.756016,5.54,...,4.67,13.560000,3.46,4.97,65.580765,102.163750,43.180275,96.351486,32.161942,91.230003
2024-11-01,29.889999,104.459999,28.450001,291.899994,9.619985,9.122093,35.610001,113.208176,48.221649,5.65,...,4.09,16.969999,3.66,4.78,61.924690,101.804054,41.267910,85.917831,33.147858,85.050003
2024-12-01,28.520000,91.930000,26.340000,295.570007,9.211755,8.149335,30.549999,106.878769,50.339321,5.02,...,3.30,16.350000,3.33,4.29,56.012188,88.777176,37.247028,83.380714,30.877611,84.309998
2025-01-01,29.780001,96.550003,24.980000,311.339996,11.209125,11.186725,46.980000,103.743843,47.448833,6.07,...,3.81,15.760000,3.13,5.07,62.216953,97.698471,32.535694,97.197182,30.462954,90.599998
2025-02-01,23.299999,90.379997,25.690001,274.209991,10.652643,9.876477,42.549999,107.276695,39.350475,6.42,...,3.19,15.930000,3.00,4.32,68.640823,95.933723,32.634258,130.307556,25.564194,86.449997
2025-03-01,21.340000,84.349998,35.619999,261.799988,12.530767,14.660862,42.599998,116.117188,33.554592,6.88,...,4.58,13.050000,2.92,6.10,77.315536,86.384468,33.452332,130.032211,25.420986,92.029999
2025-04-01,21.400000,81.669998,34.959999,274.250000,14.706901,15.792438,47.230000,114.018616,33.633984,6.70,...,4.65,13.150000,4.23,6.25,83.181679,76.190498,34.558651,117.444962,18.384472,87.820000
2025-05-01,28.059999,68.279999,37.880001,296.119995,14.696937,14.590000,40.619999,110.429489,43.857273,6.63,...,6.09,15.490000,3.58,5.85,86.582626,77.546097,33.070766,111.947868,14.841886,81.900002


In [6]:
adj_close_df.to_csv(r"D:\Tarea\adj_close_stocks.csv", index=False)

# PARTE 3
## Calculate Returns

In [7]:
# Selecionamos la data de los 6 primeros meses 
first_6_months = adj_close_df.iloc[:6]

In [8]:
# Calculamos el retorno acumulado para seleccionar las 10 acciones con mayor retorno

cumulative_returns = (first_6_months.iloc[-1] / first_6_months.iloc[0]) - 1

top_10_stocks = cumulative_returns.sort_values(ascending=False).head(10).index.tolist()

print("top 10 acciones (highest 6-month return):")
print(top_10_stocks)

top 10 acciones (highest 6-month return):
['SOUN', 'GH', 'SSRM', 'BTDR', 'AFRM', 'UPWK', 'PRVA', 'JOYY', 'SATS', 'BABA']


In [9]:
# Creamos la data de los top acciones de los ultimos 6 meses
last_6_months = adj_close_df[top_10_stocks].iloc[6:12]
last_6_months

Ticker,SOUN,GH,SSRM,BTDR,AFRM,UPWK,PRVA,JOYY,SATS,BABA
Date,,,,,,,,,,
2025-03-01,8.12,42.599998,10.030000,8.83,45.189999,13.05,22.450001,40.236801,25.580000,130.032211
2025-04-01,9.29,47.230000,10.640000,9.57,49.759998,13.15,23.480000,39.450851,22.480000,117.444962
2025-05-01,10.11,40.619999,11.830000,12.86,51.900002,15.49,22.760000,46.976967,17.730000,111.947868
2025-06-01,10.73,52.040001,12.740000,11.48,69.139999,13.44,23.000000,49.949821,27.700001,111.525024
2025-07-01,10.33,40.980000,11.950000,12.89,68.559998,11.96,19.520000,50.200001,32.590000,120.629997
2025-08-01,13.02,67.419998,19.309999,14.29,88.449600,15.39,23.040001,54.060001,61.790001,135.000000


In [10]:
# Calcular los retornos mensuales porcentuales de cada acción para los últimos 6 meses
stock_monthly_returns = last_6_months.pct_change().dropna()

print("\nRendimientos mensuales de acciones individuales (last 6 months):")
print(stock_monthly_returns)
stock_monthly_returns.to_csv(r"D:\Tarea\stock_monthly_returns.csv", index=False)


Rendimientos mensuales de acciones individuales (last 6 months):
Ticker          SOUN        GH      SSRM      BTDR      AFRM      UPWK  \
Date                                                                     
2025-04-01  0.144089  0.108685  0.060818  0.083805  0.101129  0.007663   
2025-05-01  0.088267 -0.139953  0.111842  0.343783  0.043006  0.177947   
2025-06-01  0.061325  0.281142  0.076923 -0.107309  0.332177 -0.132343   
2025-07-01 -0.037279 -0.212529 -0.062009  0.122822 -0.008389 -0.110119   
2025-08-01  0.260407  0.645193  0.615900  0.108611  0.290105  0.286789   

Ticker          PRVA      JOYY      SATS      BABA  
Date                                                
2025-04-01  0.045880 -0.019533 -0.121188 -0.096801  
2025-05-01 -0.030664  0.190772 -0.211299 -0.046806  
2025-06-01  0.010545  0.063283  0.562324 -0.003777  
2025-07-01 -0.151304  0.005009  0.176534  0.081641  
2025-08-01  0.180328  0.076892  0.895980  0.119125  


In [11]:
# Calcular los rendimientos mensuales de la cartera con ponderación  igual 
portfolio_monthly_returns = stock_monthly_returns.mean(axis=1)

print("\nRendimientos mensuales de la cartera (last 6 months):")
print(portfolio_monthly_returns)
portfolio_monthly_returns.to_csv(r"D:\Tarea\portfolio_monthly_returns.csv", index=False)


Rendimientos mensuales de la cartera (last 6 months):
Date
2025-04-01    0.031455
2025-05-01    0.052689
2025-06-01    0.114429
2025-07-01   -0.019562
2025-08-01    0.347933
dtype: float64
